In [1]:
%matplotlib notebook
import time
import numpy as np
import scipy as sp
import ipywidgets as widgets
from inspect import signature
from ipywidgets import interact, interact_manual
from IPython.display import display
from lolimot import LolimotRegressor
from plot import PlotterLLM
from functions import paraboloid

# Local Model Playground

In [2]:
# Benutzerdefnierte Funktionen
functions=dict(
    arctan=lambda u: 3 * np.arctan(2*u) + 5,
    poly=lambda u: (4+u)*(u+1)*(u-1)*(u-3)/14,
    trigonometric=lambda u: np.sin(2*np.pi*u/4) + 0.5 * np.cos(2*np.pi*u),
    affensattel=lambda u1, u2: (u1**3 - 3 * u1 * u2**2 ) / 200,
    paraboloid=paraboloid(N=3))

In [3]:
items = [widgets.ToggleButton(value=True, description='result', disabled=False, tooltip='Description'),
         widgets.ToggleButton(value=True, description='error', disabled=False, tooltip='Description'),
         widgets.ToggleButton(value=False, description='validity_function', disabled=False, tooltip='Description'),
         widgets.ToggleButton(value=False, description='models', disabled=False, tooltip='Description'),
         widgets.ToggleButton(value=False, description='report', disabled=False, tooltip='Description')]

def lolimot(model_complexity, p, x_range, func, k):
    # TRAINING
    llm = LolimotRegressor(model_complexity=model_complexity, p=p)
    
    N = len(signature(func).parameters)
    
    if N == 1:
        x = np.reshape(np.linspace(x_range[0], x_range[1], k), (k, 1))
        y = func(x).ravel()
        _ = llm.fit(x, y)
    
        # PLOT
        k = k*2
        x = np.reshape(np.linspace(x_range[0], x_range[1], k), (k, 1))
        y = func(x)
        
    elif N == 2:
        x1, x2 = np.meshgrid(np.linspace(-5, 5, k), np.linspace(-5, 5, k))
        y = np.reshape(func(x1, x2), (k*k, ))
        X = np.vstack((x1.flatten(), x2.flatten())).T
        
        _ = llm.fit(X, y)
    
        # PLOT
        k = int(np.sqrt(k) * 8)
        x1, x2 = np.meshgrid(np.linspace(-5, 5, k), np.linspace(-5, 5, k))
        y = np.reshape(func(x1, x2), (k*k, ))
        x = [x1, x2]
    else:
        x = np.meshgrid(*[np.linspace(x_range[0], x_range[1], k) for _ in range(N)])
        y = np.reshape(func(*x), (k**N, ))
        X = np.vstack((x_i.flatten() for x_i in x)).T
        
        _ = llm.fit(X, y)
        
        # PLOT
        k = int(np.sqrt(k) * 8)
        x = np.meshgrid(*[np.linspace(x_range[0], x_range[1], k) for _ in range(N)])
        y = np.reshape(func(*x), (k**N, ))

    plotter = PlotterLLM(options=[item.description for item in items if item.value])
    plotter.update(x, y, llm)

# Widgets
complexity_range = widgets.IntSlider(
    continuous_update=False, min=1, max=1000,
    layout=widgets.Layout(width='90%'),
    description="complexity")
p_range = widgets.FloatSlider(
    continuous_update=False, min=0.05, max=1.5,
    value=0.33, step=0.01, layout=widgets.Layout(width='90%'))
x_range = widgets.IntRangeSlider(
    continuous_update=False, min=-20, max=20,
    value=[-5, 5], layout=widgets.Layout(width='90%'),
    description="range")
functions = widgets.Dropdown(
    options=functions)
sample_num = widgets.IntText(value=100, description='number of samples:', disabled=False)


print("Push 'Run Interact' to start:")
plot_options = widgets.Box([widgets.Label(value="plot options:")] + items)
display(plot_options)
interact_manual(lolimot, model_complexity=complexity_range, p=p_range, x_range=x_range, func=functions, k=sample_num);

Push 'Run Interact' to start:


Box(children=(Label(value='plot options:'), ToggleButton(value=True, description='result', tooltip='Descriptio…

interactive(children=(IntSlider(value=1, continuous_update=False, description='complexity', layout=Layout(widt…